In [1]:
import pandas as pd
import joblib

In [2]:
df = pd.read_csv("csv/거주인구 분석.csv")

In [3]:
df["Month"] = df["base_date"].str[-2:]
df["Year"] = df["base_date"].str[:4]
df

,emd_nm,base_date,em_g,resid_reg_pop,foreign_pop,total_pop,Month,Year
0,건입동,2018-01,45646550,9800.0,228.0,10028.0,01,2018
1,건입동,2018-02,42953550,9814.0,224.0,10038.0,02,2018
2,건입동,2018-03,46422450,9783.0,222.0,10005.0,03,2018
3,건입동,2018-04,42438700,9788.0,229.0,10017.0,04,2018
4,건입동,2018-05,44893300,9800.0,238.0,10038.0,05,2018
...,...,...,...,...,...,...,...,...
1629,효돈동,2021-02,26128400,5289.0,65.0,5354.0,02,2021
1630,효돈동,2021-03,26845400,5262.0,61.0,5323.0,03,2021
1631,효돈동,2021-04,25200200,5267.0,59.0,5326.0,04,2021
1632,효돈동,2021-05,21724850,5266.0,60.0,5326.0,05,2021


# 각 기울기(변화량) 값 구하기

In [4]:
resident_list= list(df.columns)[3:-2]

기울기_2020_7 = []
for resident in resident_list:
    기울기_2020_7.append(df[df['base_date'] == '2020-07'][resident].values - df[df['base_date'] == '2020-06'][resident].values)
기울기_2020_8 = []
for resident in resident_list:
    기울기_2020_8.append(df[df['base_date'] == '2020-08'][resident].values - df[df['base_date'] == '2020-06'][resident].values)
 
    
len(기울기_2020_7), len(기울기_2020_7[0]), len(기울기_2020_8), len(기울기_2020_8[0])

(3, 41, 3, 41)

In [5]:
half_month_index=df[(df['Month'] == '07') |  (df['Month'] == '08') |  (df['Month'] == '09') |  (df['Month'] == '10') |  (df['Month'] == '11')|  (df['Month'] == '12')].index
half_y_rate_df=df.drop(half_month_index)
half_y_rate_df

,emd_nm,base_date,em_g,resid_reg_pop,foreign_pop,total_pop,Month,Year
0,건입동,2018-01,45646550,9800.0,228.0,10028.0,01,2018
1,건입동,2018-02,42953550,9814.0,224.0,10038.0,02,2018
2,건입동,2018-03,46422450,9783.0,222.0,10005.0,03,2018
3,건입동,2018-04,42438700,9788.0,229.0,10017.0,04,2018
4,건입동,2018-05,44893300,9800.0,238.0,10038.0,05,2018
...,...,...,...,...,...,...,...,...
1629,효돈동,2021-02,26128400,5289.0,65.0,5354.0,02,2021
1630,효돈동,2021-03,26845400,5262.0,61.0,5323.0,03,2021
1631,효돈동,2021-04,25200200,5267.0,59.0,5326.0,04,2021
1632,효돈동,2021-05,21724850,5266.0,60.0,5326.0,05,2021


In [6]:
# 각 년도별 1~6월까지의 합
year_df = half_y_rate_df.groupby(['Year','emd_nm']).sum()
year_df= year_df.reset_index()
year_df 

,Year,emd_nm,em_g,resid_reg_pop,foreign_pop,total_pop
0,2018,건입동,270542450,58767.0,1390.0,60157.0
1,2018,남원읍,250121800,115137.0,3842.0,118979.0
2,2018,노형동,1557713165,320463.0,10691.0,331154.0
3,2018,대륜동,328771500,81866.0,1082.0,82948.0
4,2018,대정읍,248583200,127135.0,9076.0,136211.0
...,...,...,...,...,...,...
151,2021,표선면,207188200,74021.0,2102.0,76123.0
152,2021,한경면,132947250,54574.0,2201.0,56775.0
153,2021,한림읍,425945474,127393.0,19672.0,147065.0
154,2021,화북동,606630650,146274.0,2066.0,148340.0


## 인덱스리스트와 food리스트로 각 값을 추출하자

### 7월

In [7]:
index_list_2021= list(year_df[year_df['Year']=='2021'].index)
index_list_2020= list(year_df[year_df['Year']=='2020'].index)
year_df_2021 = year_df[year_df['Year']=='2021']
year_df_2020 = year_df[year_df['Year']=='2020']

In [8]:
최종_변화량 = []
for r, resident in enumerate(resident_list):
    음식별_동예측리스트=[]
    for i, index in enumerate(index_list_2020):
        음식별_동예측리스트.append(기울기_2020_7[r][i] * (year_df_2021.loc[index_list_2021[i], resident]/year_df_2020.loc[index_list_2020[i], resident]))
    최종_변화량.append(음식별_동예측리스트)

In [9]:
len(최종_변화량),len(최종_변화량[0]), 최종_변화량[0][0]

(3, 41, -13.624888888888888)

In [10]:
예상_7월 = df.copy()
예상_7월 = 예상_7월[예상_7월["base_date"]=="2021-06"]
for i, index in enumerate(예상_7월.index):
    for r, resident in enumerate(resident_list):
        예상_7월.loc[index,resident]  =   max(예상_7월.loc[index,resident] + 최종_변화량[r][i], 0)
예상_7월=예상_7월.drop(['Year','Month'],axis=1)
예상_7월["base_date"]="2021-07"
예상_7월

,emd_nm,base_date,em_g,resid_reg_pop,foreign_pop,total_pop
41,건입동,2021-07,36493300,8869.375111,225.212573,9094.463330
61,구좌읍,2021-07,37807150,15433.898481,545.825995,15978.843205
103,남원읍,2021-07,62927050,18724.963993,629.259267,19353.009878
145,노형동,2021-07,239069250,55449.117766,2072.070921,57518.550763
187,대륜동,2021-07,46745900,15108.824357,193.893136,15302.807366
229,대정읍,2021-07,109850800,21989.805405,1531.981920,23518.767458
271,대천동,2021-07,56178400,13871.038922,174.141085,14043.991602
313,도두동,2021-07,19733350,3329.343622,86.930851,3415.143192
355,동홍동,2021-07,86524400,23254.077912,870.998119,24117.764949
397,봉개동,2021-07,16048700,5260.030147,77.453237,5337.001556


### 8월

In [11]:
index_list_2021= list(year_df[year_df['Year']=='2021'].index)
index_list_2020= list(year_df[year_df['Year']=='2020'].index)
year_df_2021 = year_df[year_df['Year']=='2021']
year_df_2020 = year_df[year_df['Year']=='2020']

In [12]:
최종_변화량 = []
for r, resident in enumerate(resident_list):
    음식별_동예측리스트=[]
    for i, index in enumerate(index_list_2020):
        음식별_동예측리스트.append(기울기_2020_8[r][i] * (year_df_2021.loc[index_list_2021[i], resident]/year_df_2020.loc[index_list_2020[i], resident]))
    최종_변화량.append(음식별_동예측리스트)

In [13]:
len(최종_변화량),len(최종_변화량[0]), 최종_변화량[0][0]

(3, 41, -33.089015873015875)

In [14]:
예상_8월 = df.copy()
예상_8월 = 예상_8월[예상_8월["base_date"]=="2021-06"]
for i, index in enumerate(예상_8월.index):
    for r, resident in enumerate(resident_list):
        예상_8월.loc[index,resident]  =   max(예상_8월.loc[index,resident] + 최종_변화량[r][i], 0)
예상_8월

,emd_nm,base_date,em_g,resid_reg_pop,foreign_pop,total_pop,Month,Year
41,건입동,2021-06,36493300,8849.910984,221.637719,9071.158326,06,2021
61,구좌읍,2021-06,37807150,15458.821573,538.560212,15995.732142,06,2021
103,남원읍,2021-06,62927050,18715.009002,624.098779,19337.167926,06,2021
145,노형동,2021-06,239069250,55560.051679,2034.018121,57585.358550,06,2021
187,대륜동,2021-06,46745900,15127.442624,195.679407,15323.453260,06,2021
229,대정읍,2021-06,109850800,22095.570630,1627.147968,23726.058906,06,2021
271,대천동,2021-06,56178400,13874.058383,177.570543,14051.020994,06,2021
313,도두동,2021-06,19733350,3317.085905,85.396277,3400.976135,06,2021
355,동홍동,2021-06,86524400,23241.057242,852.996506,24081.179087,06,2021
397,봉개동,2021-06,16048700,5256.021533,78.339928,5334.001089,06,2021


# 7,8월 쓰레기 예상값 

In [16]:
loaded_model = joblib.load('모델/거주인구 분석_RF.pkl')

In [17]:
X_test = 예상_7월[resident_list]
pd.options.display.float_format = '{:.6f}'.format
# pd.reset_option('display.float_format')
예상_7월_쓰레기= pd.DataFrame(loaded_model.predict(X_test))
예상_7월_쓰레기.index = 예상_7월['emd_nm']
예상_7월_쓰레기

,0
emd_nm,
건입동,36573842.500000
구좌읍,36449049.000000
남원읍,61909614.000000
노형동,233473964.000000
대륜동,50520179.500000
대정읍,88545617.500000
대천동,59029755.000000
도두동,25615910.000000
동홍동,92372766.000000


In [18]:
X_test = 예상_8월[resident_list]
pd.options.display.float_format = '{:.6f}'.format
# pd.reset_option('display.float_format')
예상_8월_쓰레기= pd.DataFrame(loaded_model.predict(X_test))
예상_8월_쓰레기.index = 예상_8월['emd_nm']
예상_8월_쓰레기

,0
emd_nm,
건입동,37622941.500000
구좌읍,36099479.000000
남원읍,61983967.500000
노형동,236493433.500000
대륜동,51910183.500000
대정읍,104344247.320000
대천동,58012356.500000
도두동,21455976.500000
동홍동,91070682.000000
